# Object Detection Manually Techniques

## Introduction

Overall our first step at Object Detection will be with manual methods, this will offer a valuable learning experience, enabling us to develop fundamental skills and gain practical insights for further exploration and advancement in the field of image processing in context of Object Detection.

These manual ways will include Background Model Initialization and Adaptive Background Subtraction with Gaussian Mixture Models (GMM)

## Background

...

...

...

## Implementation

The two ways depend on subtracting each frame from the first frame, this will lead to separate foreground objects from the background in image sequences,
the foreground objects include the objects that have moved frame by frame.

But there is a significant difference in  how the background model is initialized and updated between these approaches:

<u>**1.Background Model Initialization:**</u> initializes the background model using the first frame and calculates the foreground mask based on pixel-wise differences.

<u>**2.Adaptive Background Subtraction with Gaussian Mixture Models (GMM):**</u> uses an adaptive background subtraction algorithm based on Gaussian mixture model, which automatically updates the background model internally without explicit initialization and manual thresholding.

after that we apply a threshold to prepare the result image for object detection process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

from expt_utils import *

In [ ]:
OUT_DIR = make_out_dir('06')

FRAMES_DIR = '../dataset/frames/test/00062'

frame_files = sorted([os.path.join(FRAMES_DIR, f) for f in os.listdir(FRAMES_DIR) if f.endswith(('.jpg', '.png'))])

### Background Model Initialization

In [ ]:
bg_frame = cv2.imread(frame_files[0])

bg_frame_gray = cv2.cvtColor(bg_frame, cv2.COLOR_BGR2GRAY).astype(np.float32)

for frame_file in frame_files[1:]:

    frame = cv2.imread(frame_file)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)
    
    diff = cv2.absdiff(gray_frame, bg_frame_gray).astype(np.uint8)
    
    _, fg_mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    _, axes = plt.subplots(1, 3, figsize=(20, 8))
    axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Frame')

    axes[1].imshow(diff, cmap='gray')
    axes[1].set_title('Differential Frame')

    axes[2].imshow(fg_mask, cmap='gray')
    axes[2].set_title('Foreground Mask')

### Adaptive Background Subtraction with Gaussian Mixture Models (GMM)

In [ ]:
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

first_frame = cv2.imread(frame_files[0])
bg_subtractor.apply(first_frame)

for frame_file in frame_files:
    frame = cv2.imread(frame_file)
    
    fg_mask = bg_subtractor.apply(frame)
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Frame')

    axes[1].imshow(fg_mask, cmap='gray')
    axes[1].set_title('Foreground Mask')

### Contour Detection

In [ ]:
min_area_threshold = 200
max_area_threshold = 1000  
for frame_file in frame_files:

    frame = cv2.imread(frame_file)
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blurred_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    
    edges = cv2.Canny(blurred_frame, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    vehicles = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area_threshold < area < max_area_threshold:
            vehicles.append(contour)
    
    frame_with_vehicles = frame.copy()
    cv2.drawContours(frame_with_vehicles, vehicles, -1, (0, 255, 0), 2)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(frame_with_vehicles, cv2.COLOR_BGR2RGB))
    plt.title('Frame with Vehicles')


## Conclusion

...

...

...